In [1]:
import sys
import os
import statistics
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier
# Need to upgrade scikit-learn: 0.16.1-np110py34_0 --> 0.17-np110py34_1
from sklearn.cross_validation import cross_val_score

from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

In [10]:
# Deal with input data
def fillna(df, label):
    df_sub = df[df['Y'] == label]
    df_sub = df_sub.fillna(df_sub.median())
    return df_sub

trainX = pd.read_csv('trainingData.txt','\t', header = None)
trainX.head()
trainX.drop(trainX.columns[len(trainX.columns)-1], axis = 1, inplace = True)
trainY = pd.read_csv("trainingTruth.txt", header = None, names = ['Y'])
df = pd.concat([trainX, trainY], axis=1)
index = df.isnull().sum(axis=1) <= 1
df = df[index]

df1 = fillna(df, 1)
df2 = fillna(df, 2)
df3 = fillna(df, 3)
    
df = pd.concat([df1, df2, df3])

print(len(df))

df.head()

13813


,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,Y
5,0.6195,0.5887,0.5191,0.4036,0.6830,0.2510,1.3674,0.3906,0.7994,0.3710,...,0.3539,0.3746,-0.0527,0.9072,-0.3468,0.6580,-0.1796,1.3043,0.5133,1
7,0.5248,0.7510,0.7803,0.8005,-0.2253,-0.6152,0.4445,0.6958,0.4908,1.1152,...,0.3253,0.7496,0.9248,0.0505,-0.1132,0.8303,0.4400,-0.0496,-0.0384,1
9,0.1247,0.3009,0.5101,0.8763,0.5828,0.7488,0.7297,0.1804,-0.4469,1.1747,...,1.3754,0.9438,0.4071,0.2787,-0.0818,0.8289,0.2149,0.3401,0.4664,1
12,0.3137,0.5067,0.7877,0.6864,0.6843,1.1828,-0.4178,0.0135,1.1980,0.8184,...,-0.1579,0.2555,0.3301,0.4057,0.3577,0.9320,-0.2678,0.3455,-0.7180,1
17,-0.7778,0.7784,1.2904,0.0947,-0.1248,0.4025,-0.0519,0.0443,0.6209,0.4168,...,0.5853,0.5015,0.1034,0.1561,0.2381,0.8526,0.1803,0.3489,1.1553,1


In [4]:
X = df.iloc[:,0:-1].values
Y = df['Y'].values
X

Y_binary = np.ones((len(Y),3)) * (-1)
for i in range(3):
    index = Y == (i+1)
    Y_binary[index,i] = 1
    
testX = pd.read_csv('testData.txt','\t', header = None)
testX.drop(testX.columns[len(testX.columns)-1], axis = 1, inplace = True)
testX.fillna(testX.median(), inplace = True) # Handle NA in test data, although not necessary for this assignment.

# Build classifiers
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1, n_estimators=20)
clf3 = GaussianNB()

clf4 = DecisionTreeClassifier(max_depth=4)
clf5 = KNeighborsClassifier(n_neighbors=7)
clf6 = SVC(kernel='rbf', probability=True)
clf7 = AdaBoostClassifier(random_state=1, base_estimator=clf6, algorithm='SAMME.R')

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3),
                                     ('dt', clf4), ('kn', clf5), ('svc', clf6)], 
                        voting='soft', weights = [8, 2, 3, 1, 2, 10])

15388


In [5]:
def trainAndPredict(clf, trainX, trainY, testX, dimensionReduction = True, n_components = 30):
    
    n_train = len(trainX)
    n_test = len(testX)
    X = np.concatenate((trainX, testX), axis=0)
    
    if dimensionReduction:        
        X = preprocessing.scale(X)
        X = PCA(n_components=n_components).fit_transform(X)
    
    trainX = X[0:n_train]
    testX = X[n_train: n_train+n_test+1]

    proba = np.zeros((len(testX),3))
    
    if len(trainY.shape) > 1:
        for i in range(trainY.shape[1]):
            clf = clf.fit(trainX,trainY[:,i])
            proba[:,i] = clf.predict_proba(testX)[:,1]
    else:
        clf = clf.fit(trainX, trainY)
        proba = clf.predict_proba(testX)

    # Write to file
    results = pd.DataFrame(proba)
    results['prediction'] = np.argmax(proba, axis=1) + 1
        
    return results

In [1]:
# Print out training accuracy
results = trainAndPredict(eclf, X, Y_binary, X)
print('training accuracy',accuracy_score(Y, results['prediction']))

# Get results, write to file, and print out training accuracy
#results.to_csv('testY.txt', sep='\t', header = False, index = False)


NameError: name 'trainAndPredict' is not defined

In [ ]:
# Cross validation
from sklearn.cross_validation import ShuffleSplit,KFold

ss = ShuffleSplit(n=len(Y), n_iter=10, test_size=0.1, random_state=1)
#kf = KFold(len(Y), n_folds=10)
scores = []
for train, test in ss:
    results = trainAndPredict(clf7, X[train], Y_binary[train], X[test])
    score = accuracy_score(Y[test], results['prediction'])
    scores.append(score)
    print(score)
                  
print(np.mean(scores),np.std(scores))